In [1]:
import boto3, argparse
import sagemaker
from sagemaker.estimator import Estimator
from sagemaker.experiments.run import load_run
from sagemaker.session import Session


from sagemaker.huggingface import HuggingFaceProcessor, HuggingFace
from sagemaker.workflow.pipeline_experiment_config import PipelineExperimentConfig

from sagemaker.workflow.execution_variables import ExecutionVariables

from smexperiments import tracker
from sagemaker.utils import unique_name_from_base

ModuleNotFoundError: No module named 'boto3'

In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

/Users/mirogoettler/miniforge3/envs/ml_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=40,
    )

tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [3]:
device = "mps" if torch.backends.mps.is_available(
    ) else "gpu" if torch.cuda.is_available() else "cpu"
device

'mps'

In [5]:
model.eval()
model.to(device)
# model()
input = tokenizer(["this is a test"], return_tensors='pt')
input.to(device)
outputs = model(**input)
y_pred = torch.argmax(outputs.logits.cpu(), dim=1)



In [10]:
[i.item() for i in y_pred]

[3]

In [16]:
# session = Session(boto3.session.Session(region_name="eu-west-3"))


# with load_run(
#     sagemaker_session=session,
#     # run_name="pipelines-7tlg1lriewey-train-model-KoqVPNiyFi",
#     # experiment_name="training-pipeline",
#     ) as run:
#     run.log_metric(name="training-loss", value=100.0, step=1)

In [17]:
import pandas as pd

In [22]:
df = pd.read_csv("data/mtsamples.csv")
l = df.medical_specialty.astype(
            "category").cat.categories.values

In [23]:
l

array([' Allergy / Immunology', ' Autopsy', ' Bariatrics',
       ' Cardiovascular / Pulmonary', ' Chiropractic',
       ' Consult - History and Phy.', ' Cosmetic / Plastic Surgery',
       ' Dentistry', ' Dermatology', ' Diets and Nutritions',
       ' Discharge Summary', ' ENT - Otolaryngology',
       ' Emergency Room Reports', ' Endocrinology', ' Gastroenterology',
       ' General Medicine', ' Hematology - Oncology',
       ' Hospice - Palliative Care', ' IME-QME-Work Comp etc.',
       ' Lab Medicine - Pathology', ' Letters', ' Nephrology',
       ' Neurology', ' Neurosurgery', ' Obstetrics / Gynecology',
       ' Office Notes', ' Ophthalmology', ' Orthopedic',
       ' Pain Management', ' Pediatrics - Neonatal',
       ' Physical Medicine - Rehab', ' Podiatry',
       ' Psychiatry / Psychology', ' Radiology', ' Rheumatology',
       ' SOAP / Chart / Progress Notes', ' Sleep Medicine',
       ' Speech - Language', ' Surgery', ' Urology'], dtype=object)